# Taak - Lineair Programmeren

## PuLP

Installeer PuLP via conda in de terminal:<br>
conda install -c conda-forge pulp

In [1]:
from pulp import *
import math
import matplotlib.pyplot as plt
import numpy as np

## Beschrijving van het probleem

Veronderstel dat je 100 euro ter beschikking hebt om te investeren. Iedere dag van de week (maandag t.e.m. vrijdag), maar niet tijdens het weekend (zaterdag en zondag), kan je beslissen hoeveel geld je wilt investeren. Als je beslist om een investering te maken, <u>moet</u>
je de volgende dag 50% van de initiele investering bijleggen. De dag erna krijg je
2 keer zoveel van je initiele investering terug. Uiteindelijk wil je op zondag jouw
totale winst berekenen.
Bijvoorbeeld, een naive methode zou zijn om steeds het hoogst mogelijke bedrag te
investeren:
* Maandag: Je beslist om 66.66 euro te investeren. (33.33 euro over)
* Dinsdag: Je moet 33.33 euro bijleggen. (0 euro over)
* Woensdag: Je krijgt 133.32 euro terug. Je investeert nu 88.88 euro. (44.44 euro over)
* Donderdag: Je moet 44.44 euro bijleggen. (0 euro over)
* Vrijdag: Je krijgt 177.75 euro terug.  Je investeert nu 118.5 euro. (59.25 euro over)
* Zaterdag: Je moet 59.25 euro bijleggen. (0 euro over)
* Zondag: Je hebt nu in totaal 237 euro.

Het probleem met deze methode is dat je de opportuniteit om te investeren op dinsdag en donderdag verliest. Je beslist om dit proces te optimaliseren m.b.v. lineair programmeren.

## Rapport

Bespreek de onderstaande stellingen en vragen.

1. Stel het bovenstaand probleem op als een <i>Linear Programming</i> probleem in <u>eerste primale vorm</u>.
2. Los het probleem op gebruikmakend van PuLP.
3. Bespreek de werking van de tableaumethode in je eigen woorden.
4. Stel dat je geen ondergrens hebt voor de variabelen (dus, $\forall x\ :\ x \in \mathbb{R}$), en dus negatief geld investeren wordt mogelijk. Bestaat er nog steeds een oplossing, en zoja, startend van het nulpunt (dus, $x = 0, \forall x$), kan de tableaumethode deze vinden? Bespreek.

In [2]:

# Create problem 
prob = LpProblem("Mixed products", LpMaximize)

#days 
Investment_1 = LpVariable("Investment_1", cat='continious', lowBound=0)   
Investment_2 = LpVariable("Investment_2", cat='continious', lowBound=0)
Investment_3 = LpVariable("Investment_3", cat='continious', lowBound=0)   
Investment_4 = LpVariable("Investment_4", cat='continious', lowBound=0)
Investment_5 = LpVariable("Investment_5", cat='continious', lowBound=0)

# Objective Function:
## Profit
prob += Investment_5 * 2


# Constraints: 
prob += Investment_1 + Investment_2*0 + Investment_3*0 + Investment_4*0 + Investment_5*0   <= (2/3)*100
prob += Investment_1*(3/2) + Investment_2 + Investment_3*0 + Investment_4*0 + Investment_5*0 <= 100
prob += -Investment_1*(1/2) + Investment_2*(3/2) + Investment_3 + Investment_4*0 + Investment_5*0 <= 100
prob += -Investment_1*(1/2) - Investment_2*(1/2) + Investment_3*(3/2) + Investment_4 + Investment_5*0 <= 100
prob += -Investment_1*(1/2) - Investment_2*(1/2) - Investment_3*(1/2) + Investment_4*(3/2) + Investment_5 <= 100
prob += -Investment_1*(1/2) - Investment_2*(1/2) - Investment_3*(1/2) - Investment_4*(1/2) + Investment_5*(3/2) <= 100
prob += -Investment_1*(1/2) - Investment_2*(1/2) - Investment_3*(1/2) - Investment_4*(1/2) - Investment_5*(1/2) <= 100


#print(prob)

status = prob.solve()    # Solver 
#print(LpStatus[status])   # The solution status 
  
# Printing the final solution
print('investering van maandag:  ' ,value(Investment_1), '€')
print('investering van dinsdag:  ' ,value(Investment_2), '€')
print('investering van woensdag:  ' ,value(Investment_3), '€')
print('investering van donderdag:  ' ,value(Investment_4), '€')
print('investering van vrijdag:  ' ,value(Investment_5), '€')
print('totaal geld op zondag:  ' ,value(prob.objective), '€')



investering van maandag:   44.168734 €
investering van dinsdag:   33.746898 €
investering van woensdag:   71.46402 €
investering van donderdag:   31.761787 €
investering van vrijdag:   127.04715 €
totaal geld op zondag:   254.0943 €


# 1) Werking code
De intuïtie voor de eerste primale vorm is als volgt, eerst voegen we op de diagonaal alle investeringen toe die we zullen doen.
De matrix ziet er dan als volgt uit:
\begin{equation}
\begin{bmatrix}
x1 & 0 & 0 & 0 & 0 \\
0 & x2 & 0 & 0 & 0 \\
0 & 0 & x3 & 0 & 0 \\
0 & 0 & 0 & x4 & 0 \\
0 & 0 & 0 & 0 & x5 \\
0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}
\label{eq:aeqn}
\end{equation}
De matrix is duidelijk nog onvolledig omdat we nog geen restricties hebben opgelegd,de restricties zijn met volgende filosofie aangelegd.
* De eerste dag hebben we een restrictie van $ 100 * \frac{3}{2}$.
* De tweede dag beperken we $x2$ met $100 - \frac{3}{2} * x1$ , Dit is zo omdat we $x2$ moeten beperken door het bedrag dat de vorige dag is belegd.
* Bij de 3de dag wordt het "ritueel" duidelijk, alle vorige restricties zijn nog steeds geldig maar nu krijgen we het bedrag van de eerste dag dubbel terug. Dit betekent dat we de waarde van $x1$ er weer dubbel bijtellen terwijl we de beleggening van de dag ervoor er weer moeten aftrekken. De beperking ziet er als volgt uit $100 + \frac{1}{2} * x1 - \frac{3}{2} * x2$
Indien we de restricties doortrekken dan verkrijgen we volgende matrix met grenzen:

\begin{equation}
\begin{bmatrix}
x1 & 0 & 0 & 0 & 0 &| & \leq 100 * \frac{3}{2} \\
0 & x2 & 0 & 0 & 0 &| & \leq 100 - \frac{3}{2} * x1\\
0 & 0 & x3 & 0 & 0 &| & \leq 100 + \frac{1}{2} * x1 - \frac{3}{2} * x2 \\
0 & 0 & 0 & x4 & 0 &| & \leq 100 + \frac{1}{2} * (x1 + x2) - \frac{3}{2} * x3 \\
0 & 0 & 0 & 0 & x5 &| & \leq 100 + \frac{1}{2} * (x1 + x2 + x3) - \frac{3}{2} * x4 \\
0 & 0 & 0 & 0 & 0 &| & \leq 100 + \frac{1}{2} * (x1 + x2 + x3 + x4) - \frac{3}{2} * x5 \\
0 & 0 & 0 & 0 & 0 &| & \leq 100 + \frac{1}{2} * (x1 + x2 + x3 + x4 + x5) \\
\end{bmatrix}
\label{eq:beqn}
\end{equation}

* Omdat we onbekenden hebben in de beperkingen hebben vormen we de matrix om naar een vorm waarmee de software berekeningen kan doen

\begin{equation}
\begin{bmatrix}
x1 & 0 & 0 & 0 & 0 &| & \leq 100 * \frac{3}{2} \\
\frac{3}{2} * x1 & x2 & 0 & 0 & 0 &| & \leq 100 \\
- \frac{1}{2} * x1 & \frac{3}{2} * x2 & x3 & 0 & 0 &| & \leq 100 \\
- \frac{1}{2} * x1 & - \frac{1}{2} * x2 & \frac{3}{2} * x3 & x4 & 0 &| & \leq 100 \\
- \frac{1}{2} * x1 & - \frac{1}{2} * x2 & - \frac{1}{2} * x3 & \frac{3}{2} * x4 & x5 &| & \leq 100 \\
- \frac{1}{2} * x1 & - \frac{1}{2} * x2 & - \frac{1}{2} * x3 & - \frac{1}{2} * x4 & \frac{3}{2} * x5 &| & \leq 100 \\
- \frac{1}{2} * x1 & - \frac{1}{2} * x2 & - \frac{1}{2} * x3 & - \frac{1}{2} * x4 & - \frac{1}{2} * x5 &| & \leq 100 \\
\end{bmatrix}
\label{eq:ceqn}
\end{equation}

De variabele die we willen maximaliseren is x5, dit omdat we de laatste dag zoveel mogelijk willen beleggen zodat we zondag zoveel mogelijk geld hebben.
Bij de maximalisatie functie doen we $2 * x5$ zodat we direct de hoeveelheid geld zien, dit heeft geen invloed op het zoeken naar een maximum.
* Het uitvoeren van de bovenstaande matrix met de pulp library geeft ons een maximalisatie van 254,09€ 


# 3) De tableaumethode (Simplex methode)
Voor de simplex methode te illustreren gebruiken we een maximalistatie probleem dat we als volgt zullen opstellen.
* Een maximalisatiefunctie $max (h) = c_{1} x_{1} + c_{2} x_{2} + ... + c_{n} x_{n}$
* Een reeks constraints: $$
\begin{aligned}
  a_{1,1}x_1 + a_{1,2}x_2 + ... + a_{1,n}x_n \leq b_1\\
  a_{2,1}x_1 + a_{2,2}x_2 + ... + a_{2,n}x_n \leq b_2\\
  ...\\
  a_{m,1}x_1 + a_{m,2}x_2 + ... + a_{m,n}x_n \leq b_m\\
\end{aligned}
$$
Deze vergelijkingen staan momenteel in eerste primale vorm, om ze om te vormen om met de Simplexmethode te werken nemen we nog een extra constraint dat inhoud dat geen enkele $x_1 , x_2 , ... , x_n$ negatief mag zijn. Dit zorgt ervoor dat de oplossingsruimte van variabelen geen oneindig grote ruimte meer kan zijn, visueel meer hierover verder. Vervolgens voegen we voor elke vergelijking een "Slack variabele" toe, de vergelijkingen ziet er dan als volgt uit: \begin{aligned}
  a_{1,1}x_1 + a_{1,2}x_2 + ... + a_{1,n}x_n + S_1 = b_1\\
  a_{2,1}x_1 + a_{2,2}x_2 + ... + a_{2,n}x_n + S_2 = b_2\\
  ...\\
  a_{m,1}x_1 + a_{m,2}x_2 + ... + a_{m,n}x_n + S_m = b_m\\
\end{aligned}
De slack variabelen voegen we toe zodat we van elke inequaliteit een equaliteit kunnen maken.
Voorts demonstreren we een voorbeeld in een 2 dimensionale ruimte om de verdere intuitie van het probleem uit te leggen.
<img src="simplex_method.png"
     alt="Markdown Monster icon"
     style="float: mid; margin-right: 20px;" />
 Op de figuur zien we de 3 rechten die corresponderen met 3 lineaire vergelijkingen (van hoogstens dimensie 2) waar we de Slack variabelen op 0 hebben geinitialiseerd.
* Merk op dat x1 en x2 niet negatief mogen zijn en dus enkel het positieve kwadrant hier wordt weergegeven. <br />
Het algoritme werkt Grafisch als volgt, op de figuur die zichtbaar is kunnen we vanuit het punt $(0,0)$ 2 richtingen op. We kiezen de richting waar als we in de snijpunten van het beschikbaar gebied  komen onze maximalisatiefunctie zoveel mogelijk is gemaximaliseerd.
In dit punt kunnen we terug meerdere mogelijkheden hebben om verschillende rechten van vergelijkingen te volgen, we kiezen steeds deze waar de maximalisatiefunctie zoveel mogelijk wordt gemaximaliseerd. Naast de grafische aanpak zullen we nu de theoretische aanpak nader toelichten.
## Theoretische aanpak
Voor de theoretische aanpak stellen we een matrix op voor al deze vergelijkingen met als onderste rij de maximalisatie functie herschreven naar volgende vorm $max (h) - c_{1} x_{1} - c_{2} x_{2} - ... - c_{n} x_{n} = 0$
<img src="matrix7.png"
     alt="Markdown Monster icon"
     style="float: mid; margin-right: 20px;" />
Indien we dit in een matrix vorm schrijven dan krijgen we bovenstaand figuur. (met $ n \leq m$) <br />
De stappen die we nu doorlopen in een loop zijn als volgt
* Pak min$ (c_1 , c_2 , ... , c_n) $ (indien dit een positief getal is dan zijn we klaar), Dit is de pivotkolom dus de richting waar de max(h) functie zoveel mogelijk wordt gemaximaliseerd
* Indien er meerdere mogelijkheden zijn, dus meerdere lineaire vergelijkingen die in deze richting kunnen varieren (bv $a_{1,2} \neq 0 $ en $ a_{2,2} \neq 0$). Dan doen we volgende bewerking $\frac{b}{a}$, de waarde die het kleinste positieve resultaat geeft deze nemen we aan. De grafische intuitie hieracter is dat de grotere waarde niet meer in het aanvaardings gebied ligt dat wordt gecreëerd door de kruisende rechten. Gelijkaardig voor de negatieve waarden, deze liggen ook niet in het aanvaardingsgebied.
* Na dit doen we 2 zaken we delen de corresponderende $a_{i,j}$ door zichzelf (tenzij deze al 1 is) en delen de hele rij door dit getal. Vervolgens doen we elementaire rij operaties tot elk element in dezelfde kolom 0 is buiten de rij vanwaar $a_{i,j}$ komt.
* Voorts moeten we nu controleren of elke $c_i$ al een positief getal is, indien dit niet het geval is herhalen we de loop weer, anders hebben we het antwoord
Indien we het antwoord hebben hoeven we dit nog op de correcte manier te interpreteren, een antwoord kan er als volgt uitzien.
<img src="matrix5.png"
     alt="Markdown Monster icon"
     style="float: mid; margin-right: 20px;" />
We negeren het feit dat sommige waarden hetzelfde zijn gebleven als vorige matrix, tussen $g_1$ tot $g_m$ vinden we de respectievelijke waarden voor $x_1$ tot $x_n$.
Op de laatste rij vinden we in de constraint kolom de maximalisatie van onze functie.
Wat natuurlijk de antwoord op de intiele vraag van het probleem was.
# 4) Tableau methode zonder ondergrens
1 van de vereisten zodat de tableaumethode werkt is dat elke variabele positief is, de tableaumethode zou hier dus geen oplossing voor kunnen vinden.


# Referenties 
* https://www.youtube.com/watch?v=jh_kkR6m8H8
* https://ece.uwaterloo.ca/~dwharder/NumericalAnalysis/16LP/simplex/theory.html
